In [2]:
!pip install -q torch_geometric

# Optional dependencies:
!pip install -q pyg_lib torch_scatter torch_sparse torch_cluster \
torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

ERROR: Could not find a version that satisfies the requirement pyg_lib (from versions: none)
ERROR: No matching distribution found for pyg_lib


In [2]:
import os
import torch

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.1.0+cu118


In [3]:
# Import all required modules:
import scipy.io as sc
import os
import torch
import networkx as nx
import matplotlib.pyplot as plt
import csv
import pandas as pd
import re
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, GraphConv
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool
import random
from sklearn.model_selection import KFold

#### Rewrite the .mat files into a useable format:
Then save the data into useable variables

In [4]:
%cd ./Documents/Git_coding_respositories/ECE_Project_2/Deep-learn

[WinError 3] The system cannot find the path specified: './Documents/Git_coding_respositories/ECE_Project_2/Deep-learn'
c:\Users\Adam\Documents\Git_Coding_respositories\ECE_Project_2\Deep-learn


In [13]:
%pwd

'C:\\Users\\Adam\\Documents\\Git_coding_respositories\\ECE_Project_2\\Deep-learn'

In [5]:
# Make directory if it doesn't exist
if os.path.exists("./data_processed/BP") == False:
    os.makedirs("./data_processed/BP")
    os.makedirs("./data_processed/NC")
    os.makedirs("./data_processed/All")

file_root = "./data"
file_root_data_path = os.listdir(file_root)
bp = []

for file in file_root_data_path:
    mat1 = sc.loadmat("{}/{}".format(file_root, file))
    mat1 = np.array(mat1["C"])
    # Process each file type into separate folders in case needed
    if re.search("BP", file):
        with open("./data_processed/BP/{}.txt".format(os.path.splitext(file)[0]), "w") as f:
            csv.writer(f, delimiter=',').writerows(mat1)
    elif re.search("NC", file):
        with open("./data_processed/NC/{}.txt".format(os.path.splitext(file)[0]), "w") as f:
            csv.writer(f, delimiter=',').writerows(mat1)
    # Saved all files into an all folder
    with open("./data_processed/All/{}.txt".format(os.path.splitext(file)[0]), "w") as f:
        csv.writer(f, delimiter=',').writerows(mat1)
        
# Load the adjacency matrices from the text files as numpy arrays:
import pandas as pd

bp_root = "./data_processed/BP"
bp_root_path = os.listdir(bp_root)
nc_root = "./data_processed/NC"
nc_root_path = os.listdir(nc_root)
all_files_root = "./data_processed/All"
all_files_root_path = os.listdir(all_files_root)
bp = []
nc = []
all_files = []

for file in bp_root_path:
    mat1 = pd.read_csv("{}/{}".format(bp_root,file), header=None).to_numpy()
    bp.append(mat1)

for file in nc_root_path:
    mat1 = pd.read_csv("{}/{}".format(nc_root,file), header=None).to_numpy()
    nc.append(mat1)

labels = []

for file in all_files_root_path:
    mat1 = pd.read_csv("{}/{}".format(all_files_root,file), header=None).to_numpy()
    if re.search("BP", file):
        labels.append(1)
    else:
        labels.append(0)
    all_files.append(mat1)

### Data Setup:
Rewrite .mat files into a usable format, Load those files into variables:

In [6]:
# Convert the adjacency matrices to dataframes that can be used in pytorch:
# Function to calculate the edge_matrix
def calc_edge_matrix(adjacency_matrices):
        num_nodes = adjacency_matrices[0].shape[0]
        edge_index = []
        for i in range(num_nodes):
            for j in range(i + 1, num_nodes):
                weight = all_files[i][j]
                if weight.any != 0:
                    edge_index.append([i, j])
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        return edge_index

# Prepare Data for PyTorch
def prepare_data(adjacency_matrices, labels):
    graph_data = []
    for adj_matrix, label in zip(adjacency_matrices, labels):
        edge_index = calc_edge_matrix(adjacency_matrices)
        x = torch.tensor((adj_matrix), dtype=torch.float)  # Features (e.g., all ones)
        y = torch.tensor([label], dtype=torch.long)
        num_classes = 2
        num_features = len(bp[0])
        num_nodes = len(bp[0])
        num_edges = 3486
        data = Data(x=x, edge_index=edge_index, y=y, num_classes=num_classes, num_features = num_features, num_nodes = num_nodes, num_edges = num_edges)
        graph_data.append(data)
    return graph_data
# Create a dataset from the prepared data
graph_data = prepare_data(all_files, labels)

# Perform the K-fold cross-validation splits:
k = 5 # number of spits to perform, must be 5 for 80/20
kf = KFold(n_splits=k, shuffle=True, random_state=499)
random.seed(412)
shuffled_graph_data = random.sample(graph_data, len(graph_data))
train_mask = []
test_mask = []

# Generate the train and test mask for selecting the data:
for i, (train_index, test_index) in enumerate(kf.split(shuffled_graph_data)):
    train_mask.append(train_index)
    test_mask.append(test_index)

train_data = [[] for _ in range(5)]
for i in range(len(train_mask)):
    for j in train_mask[i]:
        train_data[i].append(shuffled_graph_data[j])

test_data = [[] for _ in range(5)]
for i in range(len(test_mask)):
    for j in test_mask[i]:
        test_data[i].append(shuffled_graph_data[j])

train_loader = DataLoader(train_data, batch_size=7, shuffle=False)
test_loader = DataLoader(test_data, batch_size=2, shuffle=False)

c:\Users\Adam\anaconda3\Lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [7]:
# Grab information about the dataset:
print()
print(f'Dataset: {graph_data[0]}:')
print('====================')
print(f'Number of graphs: {len(graph_data)}')
print(f'Number of features: {graph_data[0].num_features}')
print(f'Number of classes: {graph_data[0].num_classes}')

data = graph_data[0]

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')


Dataset: Data(x=[84, 84], edge_index=[2, 3486], y=[1], num_classes=2, num_features=84, num_nodes=84, num_edges=3486):
Number of graphs: 97
Number of features: 84
Number of classes: 2

Data(x=[84, 84], edge_index=[2, 3486], y=[1], num_classes=2, num_features=84, num_nodes=84, num_edges=3486)
Number of nodes: 84
Number of edges: 3486
Average node degree: 41.50


### Build and Instantiate the GNN

In [45]:
from torch.nn import Conv1d, BatchNorm1d, MaxPool1d, Softmax2d, Conv3d

In [54]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, output_channels):
        super(GCN, self).__init__()
        torch.manual_seed(23145)
        self.conv1 = Conv1d(84, hidden_channels, 1)
        self.conv2 = BatchNorm1d(84, hidden_channels)
        self.conv3 = MaxPool1d(hidden_channels, hidden_channels)

        self.conv4 = Conv1d(hidden_channels, hidden_channels, 1)
        self.conv5 = BatchNorm1d(hidden_channels, 1)
        self.conv6 = MaxPool1d(hidden_channels, hidden_channels)

        self.conv7 = Conv1d(hidden_channels, hidden_channels, 1)
        self.conv8 = BatchNorm1d(hidden_channels, hidden_channels)
        self.conv9 = MaxPool1d(hidden_channels, hidden_channels)

        # Create classification
        self.fc1 = Linear(hidden_channels, hidden_channels)
        self.fc2 = Linear(hidden_channels, output_channels)
        self.sm = Softmax2d()
        self.lin = Linear(output_channels, 2)

    def forward(self, x, edge_index):
        # 1. Obtain node embeddings
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.relu()
        x = self.conv3(x)
        
        x = self.conv4(x)
        x = self.conv5(x)
        x = x.relu()
        x = self.conv6(x)

        x = self.conv7(x)
        x = self.conv8(x)
        x = x.relu()
        x = self.conv9(x)

        # Create classification head:
        x = self.fc1(x)
        x = x.relu()
        x = self.fc2(x)
        x = self.sm(x)
        x = self.lin(x)

        return x

model = GCN(hidden_channels=64, output_channels = 64)
print(model)

# Define train and test functions
def train(loader, k):
    model.train()

    for data in loader.dataset[k]:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader, k):
    model.eval()

    correct = 0
    for data in loader.dataset[k]:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset[k]) # Derive ratio of correct predictions.

GCN(
  (conv1): Conv1d(84, 64, kernel_size=(1,), stride=(1,))
  (conv2): BatchNorm1d(84, eps=64, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): MaxPool1d(kernel_size=64, stride=64, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  (conv5): BatchNorm1d(64, eps=1, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): MaxPool1d(kernel_size=64, stride=64, padding=0, dilation=1, ceil_mode=False)
  (conv7): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  (conv8): BatchNorm1d(64, eps=64, momentum=0.1, affine=True, track_running_stats=True)
  (conv9): MaxPool1d(kernel_size=64, stride=64, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (sm): Softmax2d()
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


### Train and then evaluate the GNN:

In [55]:
master_train_acc = []
master_test_acc = []

#=========== Loop variables ========================
counter = k # Number of k-fold splits being performed
num_epochs = 35

# Model variables
hidden_channels = 64
output_channels = 64
#===================================================
for i in range(counter):
    train_acc_history = []
    test_acc_history = []
    model = GCN(hidden_channels, output_channels)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()
    if i == 0:
        print(f"==================== K-fold data split {i+1} ====================")
    else:
        print(f"\n\n==================== K-fold data split {i+1} ====================")
    for epoch in range(num_epochs+1):
        train(train_loader, i)
        train_acc = test(train_loader, i)
        train_acc_history.append(train_acc)
        test_acc = test(test_loader, i)
        test_acc_history.append(test_acc)
        print(f'Epoch: {epoch:03d}, Train Acc: {100*train_acc:.4f}%, Test Acc: {100*test_acc:.4f}%')

    master_train_acc.append(train_acc)
    master_test_acc.append(test_acc)

    plt.figure(figsize=(7,4))
    plt.title(f"K-fold data split {i+1}")
    plt.plot(range(1, num_epochs+2), train_acc_history, label='Training Accuracy')
    plt.plot(range(1, num_epochs+2), test_acc_history, label='Testing Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

print(f"The overall average training accuracy is: {np.mean(master_train_acc):.4f}")
print(f"The overall average testing accuracy is: {np.mean(master_test_acc):.4f}")


==================== K-fold data split 1 ====================


RuntimeError: running_mean should contain 1 elements not 64

#### Perform Light probability analysis to determine if the model was just getting lucky:

Basic probability analysis: Is the model simply getting lucky with random guesses?


In [24]:
# Probability analysis -- are my results entirely from chance?
y = []
ones = 0
zeros = 0
for i in range(len(test_data)):
        for item in test_data[i]:
                y.append(item.y.tolist())
                if item.y == 1:
                        ones += 1
                else:
                        zeros += 1
        print(f"For dataset {i+1}:")
        print(f"The random chance that the model predicts the ones class: {ones / len(y)}")
        print(f"The random chance that the model precicts the zeros class: {zeros/ len(y)}")
        print("============================================================")

For dataset 1:
The random chance that the model predicts the ones class: 0.3
The random chance that the model precicts the zeros class: 0.7
For dataset 2:
The random chance that the model predicts the ones class: 0.425
The random chance that the model precicts the zeros class: 0.575
For dataset 3:
The random chance that the model predicts the ones class: 0.5254237288135594
The random chance that the model precicts the zeros class: 0.4745762711864407
For dataset 4:
The random chance that the model predicts the ones class: 0.5512820512820513
The random chance that the model precicts the zeros class: 0.44871794871794873
For dataset 5:
The random chance that the model predicts the ones class: 0.5360824742268041
The random chance that the model precicts the zeros class: 0.4639175257731959
